In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model


In [2]:
dataset, info = tfds.load("mnist", with_info = True, as_supervised= True)

Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/1 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/2 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/3 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/4 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Dl Completed...:   0%|          | 0/4 [00:01<?, ? url/s]
Dl Size...: 0 MiB [00:01, ? MiB/s]

Dl Size...:   0%|          | 0/9 [00:01<?, ? MiB/s]

Dl Size...:   0%|          | 0/9 [00:01<?, ? MiB/s]

Dl Size...:   0%|          | 0/10 [00:01<?, ? MiB/s]

Dl Size...:   0%|          | 0/10 [00:01<?, ? MiB/s]

Dl Size...:   0%|          | 0/10 [00:01<?, ? MiB/s]

Extraction completed...:   0%|          | 0/1 [00:01<?, ? file/s]

Dl Size...:   0%|          | 0/10 [00:01<?, ? MiB/s]

Dl Size...:   0%|          | 0/10 [00:01<?, ? MiB/s]

Dl Size...:   0%|          | 0/10 [00:01<?, ? MiB/s]

Extraction completed...:  50%|█████     | 1/2 [00:01<00:01,  1.08s/ file]

Dl Size...:   0%|          | 0/10 [00:01<?, ? MiB/s]

Dl Size...:  10%|█         | 1/10 [00:06<00:55,  6.15s/ MiB]

Dl Size...:  20%|██        | 2/10 [00:07<00:36,  4.58s/ MiB]

Dl Size...:  20%|██        | 2/10 [00:10<00:36,  4.58s/ MiB]

Dl Size...:  20%|██        | 2/10 [00:10<00:36,  4.58s/ MiB]

Extraction completed...:  67%|███

60000 examples [00:15, 3852.77 examples/s]
Shuffling...:   0%|          | 0/10 [00:00<?, ? shard/s]WARNING: Logging before flag parsing goes to stderr.
W0408 15:26:53.106928 4349789632 deprecation.py:323] From /anaconda3/lib/python3.6/site-packages/tensorflow_datasets/core/file_format_adapter.py:249: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`

Reading...: 0 examples [00:00, ? examples/s]
Reading...: 6000 examples [00:00, 190344.48 examples/s]
Writing...: 100%|██████████| 6000/6000 [00:00<00:00, 177184.18 examples/s]
Reading...: 0 examples [00:00, ? examples/s]
Reading...: 6000 examples [00:00, 351595.84 examples/s]
Shuffling...:  20%|██        | 2/10 [00:00<00:00, 14.23 shard/s]
Reading...: 0 examples [00:00, ? examples/s]
Reading...: 6000 examples [00:00, 220030.99 examples/s]
Writing...: 100%|██████████| 6000/6000 [00:00<00:00, 

In [4]:
mnist_train, mnist_test = dataset["train"], dataset["test"]

In [6]:
def convert_types(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

In [9]:
mnist_train = mnist_train.map(convert_types).shuffle(10000).batch(32)
mnist_test = mnist_test.map(convert_types).batch(32)

In [13]:


class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation = 'relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation = 'relu')
        self.d2 = Dense(10, activation = 'softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

In [15]:
model = MyModel()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [16]:

train_loss = tf.keras.metrics.Mean(name = "train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = "train_accuracy")

test_loss = tf.keras.metrics.Mean(name = "test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = "test_accuracy")


In [18]:

@tf.function
def train_step(image, label):
    with tf.GradientTape() as tape:
        predictions = model(image)
        loss = loss_object(label, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    
    train_loss(loss)
    train_accuracy(label, predictions)




In [19]:
@tf.function
def test_step(image, label):
    predictions = model(image)
    t_loss = loss_object(label, predictions)
    
    test_loss(t_loss)
    test_accuracy(label, predictions)


In [20]:

EPOCHS = 5

for epoch in range(EPOCHS):
    for image, label in mnist_train:
        train_step(image, label)
        
    for test_image, test_label in mnist_test:
        test_step(test_image, test_label)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))


Epoch 1, Loss: 0.1305551677942276, Accuracy: 96.02833557128906, Test Loss: 0.056781403720378876, Test Accuracy: 98.19999694824219
Epoch 2, Loss: 0.08641991019248962, Accuracy: 97.37166595458984, Test Loss: 0.059862345457077026, Test Accuracy: 98.0999984741211
Epoch 3, Loss: 0.06529910117387772, Accuracy: 97.99833679199219, Test Loss: 0.05828314647078514, Test Accuracy: 98.15333557128906
Epoch 4, Loss: 0.05232755094766617, Accuracy: 98.38667297363281, Test Loss: 0.05865881219506264, Test Accuracy: 98.1624984741211
Epoch 5, Loss: 0.043777428567409515, Accuracy: 98.64266204833984, Test Loss: 0.06122233346104622, Test Accuracy: 98.1760025024414
